In [ ]:
from models.helper import GetModel
from dataloaders.train_loader import GSVCitiesDataset
from dataloaders.test_loader import TestDataset
from torch.utils.data import DataLoader
import torch
import os
from pytorch_metric_learning import losses, miners
from train import train_model
from evaluation import eval_model
from utils.lr_scheduler import custom_scheduler
import random
import numpy as np
import torch

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)


train_dataset = GSVCitiesDataset(generated_data_prob=0)
test_dataset = TestDataset()

num_workers = 16
train_dataloader = DataLoader(train_dataset, 
                              batch_size=100, 
                              shuffle=True, 
                              num_workers=num_workers, 
                              pin_memory=True
                              )

test_dataloader = DataLoader(test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )
sf_xs_test_dataset = TestDataset(path='./data/sf_xs/test')
sf_xs_test_dataloader = DataLoader(sf_xs_test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )
tokyo_test_dataset = TestDataset(path='./data/tokyo_xs/test')
tokyo_test_dataloader = DataLoader(tokyo_test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )


# Phase 1

### Experiment 1: Avg Pooling + Contrastive Loss + No Miner + SGD(lr=0.01, decay=1e-3)

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'contrastive_avg_model'

model = GetModel(aggregator='average')
model = model.to('cuda')
if TRAIN:   
    miner = None
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

### Experiment 2: Gem Pooling + Contrastive Loss + No Miner + SGD(lr=0.01, decay=1e-3)

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'contrastive_gem_model'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = None
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

# Phase 2

#### Experiment 2.1: Batch Hard Miner + Contrastive Loss

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'contrastive_batchhard_model'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

#### Experiment 2.2: DistanceWeightedMiner + SupConLoss


In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'supcon_distanceweighted_model'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.DistanceWeightedMiner()
    loss_fn = losses.SupConLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

#### Experiment 2.3: PairMargin + FastAP

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'fastap_pairmargin_model'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.PairMarginMiner()
    loss_fn = losses.FastAPLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

#### Experiment 2.4: No Miner + MultiSimilarityLoss

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'multisimilarity_model'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = None
    loss_fn = losses.MultiSimilarityLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

# Phase 3: Optimizer, LR, WD and LR Scheduler

## SGD, 1e-2, 1e-5

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'SGD_1e2_1e5'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-5,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

## 3.2 ADAMW, 1e-4, 1e-4

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'AdamW_1e4_1e4'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.AdamW(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-4)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

## 3.3 ADAM, 1e-4, 1e-5

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

# LR SCHEDULER

##nO

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5_no_scheduler'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

## Poly

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5_custom'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

## Cosine

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5_cosine'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'cosine',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

# MIXVPR

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5_mixvpr_custom'

model = GetModel(aggregator='mixvpr', input_size=256, output_size=512)
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'patience': 45,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5_mixvpr_1024_custom'

model = GetModel(aggregator='mixvpr', input_size=256, output_size=1024)
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'patience': 45,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

# Phase 4: Depth

In [1]:
from models.helper import GetModel, GetMultiModel, GetDeAttNet
from dataloaders.train_loader import GSVCitiesDataset
from dataloaders.test_loader import TestDataset
from torch.utils.data import DataLoader
import torch
import os
from pytorch_metric_learning import losses, miners
from train import train_model
from evaluation import eval_model
from utils.lr_scheduler import custom_scheduler
import random
import numpy as np
import torch

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)


train_dataset = GSVCitiesDataset(generated_data_prob=0, multi_model=True)
test_dataset = TestDataset(multi_model=True)

num_workers = 16
train_dataloader = DataLoader(train_dataset, 
                              batch_size=100, 
                              shuffle=True, 
                              num_workers=num_workers, 
                              pin_memory=True
                              )

test_dataloader = DataLoader(test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )
sf_xs_test_dataset = TestDataset(path='./data/sf_xs/test', multi_model=True)
sf_xs_test_dataloader = DataLoader(sf_xs_test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )
tokyo_test_dataset = TestDataset(path='./data/tokyo_xs/test', multi_model=True)
tokyo_test_dataloader = DataLoader(tokyo_test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )


torch.cuda.empty_cache()

# Concat

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_depth_no_fusion_arda_512'

model = GetMultiModel(aggregator='gem', use_fusion=False, output_size=512)
model = model.to('cuda')
# model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_depth_no_fusion_arda_1024'

model = GetMultiModel(aggregator='gem', use_fusion=False, output_size=1024)
model = model.to('cuda')
# model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()

# Depth+ mixvpr

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_depth_mixvpr_arda_512'

model = GetMultiModel(aggregator='mixvpr', use_fusion=False, input_size=512, output_size=512)
model = model.to('cuda')
# model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_depth_mixvpr_arda_1024'

model = GetMultiModel(aggregator='mixvpr', use_fusion=False, input_size=512, output_size=1024)
model = model.to('cuda')
# model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()

# DEATTNET

In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_deattnet'

model = GetDeAttNet(aggregator='gem')
model = model.to('cuda')
# model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()